In [26]:
#%pip install pandas
#%pip install seaborn


In [2]:

import pandas as pd
import numpy as np
import seaborn as sns
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


In [3]:
os.getcwd()

'/workspaces/mlops-zoomcamp/01-intro'

In [4]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")

- Read the data for January. How many columns are there?



In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
len(df.columns)

19

- There are 19 columns.

In [7]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

# Q2. Computing duration



- Now let's compute the duration variable. It should contain the duration of a ride in minutes.

- What's the standard deviation of the trips duration in January?



In [8]:
df["duration"] = df["tpep_dropoff_datetime"]-df["tpep_pickup_datetime"]
np.std(df["duration"])

Timedelta('0 days 00:42:35.660657')

- Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

- What fraction of the records left after you dropped the outliers?

In [9]:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


In [10]:
df["duration"]

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [11]:
len(df[(df.duration >= 1) & (df.duration <= 60)])/len(df)

0.9812202822125979

- What fraction of the records left after you dropped the outliers?

- 90%

- 92%

- 95%

- **98%**

In [12]:
df = df[(df.duration >= 1) & (df.duration <= 60)]


# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.



- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)


In [13]:
columns_with_id = [col for col in df.columns if 'LocationID' in col]

df[columns_with_id] = df[columns_with_id].astype(str)

In [14]:
columns_with_id

['PULocationID', 'DOLocationID']

In [ ]:
#features = pd.get_dummies(df[columns_with_id], drop_first=True)

: 

In [15]:
df[columns_with_id].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [16]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [17]:
dv = DictVectorizer()

In [18]:
#features

In [19]:
train_dicts = df[categorical + numerical].to_dict(orient='records')


In [20]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.658396898909143

In [21]:
X_train

<3009173x516 sparse matrix of type '<class 'numpy.float64'>'
	with 9027519 stored elements in Compressed Sparse Row format>

**What's the dimensionality of this matrix (number of columns)?**

- 2
- 155
- 345
- 515
- 715


In [22]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [23]:
df_val = read_dataframe('../data/yellow_tripdata_2023-02.parquet')


In [24]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df[target].values
y_val = df_val[target].values

: 

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

NameError: name 'LinearRegression' is not defined